In [1]:
import json
import s2sphere as s2
from copy import deepcopy

In [2]:
with open('collegeonly.json', 'r', encoding='utf8') as read_file:
    collegecells = json.load(read_file)

In [3]:
with open('univonly.json', 'r', encoding='utf8') as read_file:
    univcells = json.load(read_file)

In [4]:
with open('geonames.json', 'r', encoding='utf8') as read_file:
    geonames = json.load(read_file)

In [5]:
with open('usunivandcollegeindex.json', 'r', encoding='utf8') as read_file:
    usindex = json.load(read_file)['elements']

In [6]:
def convertToTokenDict(cell_list):
    ldict = {}
    for element in cell_list:
        for token in element['cover16s']:
            if token in ldict:
                raise ValueError
            else:
                ldict[token] = element
    return ldict

In [7]:
univldict = convertToTokenDict(univcells)

In [8]:
collegeldict = convertToTokenDict(collegecells)

In [9]:
len(univcells)

In [10]:
len(collegecells)

In [11]:
def findspecificcode(namelist):
    ftcode = []
    for el in namelist:
        if el['feature-code'] == 'UNIV':
            ftcode.append(el)
    return ftcode

In [12]:
univonlygeonames = findspecificcode(geonames)

In [13]:
len(univonlygeonames)

In [14]:
len(usindex)

In [15]:
def linkCoordinateToBound(ldicts, namelist, numexpansions=0):
    matched = 0
    unmatched = 0
    matchedindex = []
    unmatchedindex = []
    def expandTokens(token):
        instcell = s2.CellId.from_token(token)
        nbrs = set(map(lambda x: x.to_token(), list(instcell.get_edge_neighbors()) + [instcell]))
        return nbrs
    
    for ldict in ldicts:
        for ltok in ldict:
            if ldict[ltok].get('matched'):
                ldict[ltok].pop('matched')
    
    
    for institution in namelist:
        insttok = s2.CellId.from_lat_lng(s2.LatLng.from_degrees(float(institution['latitude']), float(institution['longitude']))).parent(16).to_token()
        
        allnbrs = set([insttok])
        allmmbrs = set()
        
        for expindex in range(0,numexpansions+1):
            for celltok in list(allnbrs):
                if celltok not in allmmbrs:
                    if expindex != numexpansions:
                        thenbrs = expandTokens(celltok)
                        allnbrs.update(thenbrs)
                    allmmbrs.add(celltok)
  
        somematch = False
        for ldict in ldicts:
            for exptok in allmmbrs:
                coorsbound = ldict.get(exptok)
                if coorsbound:
                    if ldict[exptok].get('matched'):
                        ldict[exptok]['matched'].append(institution)
                    else:
                        ldict[exptok]['matched'] = [institution]
                    somematch = True
                    break
                    
        if somematch:
            matchedindex.append(institution)
            matched += 1
        else:
            unmatchedindex.append(institution)
            unmatched += 1
        
    print(matched)
    print(unmatched)
    return matchedindex, unmatchedindex
        

In [16]:
matched, unmatched = linkCoordinateToBound([univldict, collegeldict], usindex, numexpansions=2)

In [18]:
from nltk.corpus import stopwords
import string

In [19]:
# import nltk
# nltk.download("stopwords")

In [20]:
exclude = set(string.punctuation)

In [21]:
exclude.update(list(stopwords.words('english')))

In [22]:
exclude.add('college')
exclude.add('university')

In [23]:
exclude.add('')

In [ ]:
thecount = 0
for lst in [univcells, collegecells]:
    for el in lst:
        for nm in unmatched:
            splital = [tt for tt in (nm['alias'].lower().split(' ')) if tt not in exclude]
            
            splitnm = [tt for tt in (nm['institution'].lower().split(' ')) if tt not in exclude]
            
            splitnm.extend(splital)
            
            
            hhh=[gg.lower() for gg in el['allnames']]
            uuu = []
            for jgj in hhh:
                uuu.extend(jgj.split(' '))
            splitallnm = [bb for bb in hhh if bb not in exclude]
            
            print(splitnm)
            print(splitallnm)
            print()

In [31]:
thecount

In [32]:
thecount

In [25]:
for el in univcells:
    if 'matched' not in el:
        print(el['allnames'])
        print('\n')

In [24]:
for el in univcells:
    if 'matched' in el:
        if len(el['matched']) > 1:
            print(el['allnames'])
            for matchedel in el['matched']:
                print(matchedel['institution'])
            print('\n')

In [55]:
for name in formalnames:
    if 'Berkeley' in name['name'] and 'University of California' in name['name']:
        print(name)

In [59]:
s2.CellId.from_lat_lng(s2.LatLng.from_degrees(37.87215, -122.25975)).parent(16).to_token()

In [ ]:
exp1 = linkCoordinateToBound(locdict, formalnames, numexpansions=1)

In [ ]:
exp2 = linkCoordinateToBound(locdict, formalnames, numexpansions=2)

In [ ]:
#0expansion:
# 74232
# 211192

#1expansion:
# 83678
# 201746

#3expansions
# 99244
# 186180

In [26]:
b = set()

In [39]:
len(locdict)